In [8]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO
import os

In [2]:
account_name = 'bs0tfm'
container_name = 'landing'
folder_prefix = 'unprocessed/input/'
carpeta_destino = 'unprocessed/auxiliar/'
carpeta_procesada = 'processed/'

In [3]:
account_url = f"https://{account_name}.blob.core.windows.net"

In [ ]:
# Mala práctica, pero no consigo habilitar el acceso a la identidad administrada desde el pool de spark
connection_string = 'borrado'

In [5]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [6]:
excel_files = []

for blob in container_client.list_blobs(name_starts_with=folder_prefix):
    if (blob.name.endswith('xlsx')) or (blob.name.endswith('xls')):
            excel_files.append(blob.name)

In [7]:
excel_files

In [21]:
for file in excel_files:
    excel_blob = container_client.download_blob(file)
    excel_bytes = BytesIO(excel_blob.readall()) # readall es para leer todos los datos, no tiene nada que ver con las sheets
    # Por ahora, el que estemos con un excel no nos afecta en nada, claro
    xls = pd.ExcelFile(excel_bytes)
    hojas = xls.sheet_names
    solo_name = os.path.basename(file)

    for hoja in hojas:
        df = xls.parse(sheet_name = hoja)
        csv_bytes = df.to_csv(index = False).encode('utf-8')
        csv_name = f'{carpeta_destino}{solo_name}_{hoja}.csv'
        container_client.upload_blob(name = csv_name, data = BytesIO(csv_bytes), overwrite = True)
    copia_name = f'{carpeta_procesada}{solo_name}'
    #original_name = f'{folder_prefix}{file}'
    container_client.upload_blob(name = copia_name, data = excel_bytes, overwrite = True)
    #processed_files = []
    #for copia in container_client.list_blobs(name_starts_with=carpeta_procesada):
    #    processed_files.append(copia.name)
    #if copia_name in processed_files:
    #    blob_a_borrar = container_client.get_blob_client(blob = file)
    #    blob_a_borrar.delete_blob()